# Cálculo de quantidade de papéis a serem adquiridos

A ideia deste trabalho é realizar um cálculo de quantidade de papéis a serem adquiridos a partir de um montante inicial a ser investido e considerando apenas uma lista de ativos previamente avaliados e selecionados para composição de uma carteira teórica.

Para isso será utilizada técnica de Pesquisa Operacional (_Operational Research_ ou _OR_) e programação linear (_Linear Programming_ ou _LP_).

## Importando o módulo `pywraplp` da biblioteca `ortools.linear_solver`

In [76]:
from ortools.linear_solver import pywraplp

## Inicializando o `solver`

In [77]:
solver = pywraplp.Solver(
    "Maximizar proventos",
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING,
    # pywraplp.Solver. ,
)

## Definindo variáveis de decisão

Aqui achei interessante utilizar o _ticker_ dos papéis como nome da variável para facilitar a referenciação. A variável com o nome do _ticker_ representará a quantidade do mesmo, que é a resposta que buscamos a fim de maximizar o valor final estimado de proventos mensais -- tomando como base o _Dividend Yeld_ (_DY_) médio dos últimos 12 meses, ciente que os resultados passados não são garantia para o futuro.

In [78]:
# quantidade mínima # quantidade de papéis que já possui
FATN11_min_qtd =  62 # era 0
HGLG11_min_qtd =  35 # era 0
HGRE11_min_qtd =  45 # era 0
HTMX11_min_qtd =  33 # era 0
RBRP11_min_qtd = 102 # era 0
TVRI11_min_qtd =  56 # era 0
VISC11_min_qtd =  48 # era 0
XPIN11_min_qtd =  76 # era 0
XPLG11_min_qtd =  54 # era 0

# LLLL## = quantidade de papéis
FATN11 = solver.IntVar(FATN11_min_qtd, solver.infinity(), "FATN11")
HGLG11 = solver.IntVar(HGLG11_min_qtd, solver.infinity(), "HGLG11")
HGRE11 = solver.IntVar(HGRE11_min_qtd, solver.infinity(), "HGRE11")
HTMX11 = solver.IntVar(HTMX11_min_qtd, solver.infinity(), "HTMX11")
RBRP11 = solver.IntVar(RBRP11_min_qtd, solver.infinity(), "RBRP11")
TVRI11 = solver.IntVar(TVRI11_min_qtd, solver.infinity(), "TVRI11")
VISC11 = solver.IntVar(VISC11_min_qtd, solver.infinity(), "VISC11")
XPIN11 = solver.IntVar(XPIN11_min_qtd, solver.infinity(), "XPIN11")
XPLG11 = solver.IntVar(XPLG11_min_qtd, solver.infinity(), "XPLG11")

## Variáveis

In [79]:
## variáveis

valor_maximo = 41_000 + 44_744.61 # era 53_541.66
divisao = 9
fator = 0.10 / 2  # +ou- 10%

valor_equi_min = (valor_maximo / divisao) * (1 - fator)
valor_equi_max = (valor_maximo / divisao) * (1 + fator)

## Restrições

In [80]:
# preços
FATN11_prc =  75.15 # era   89.58
HGLG11_prc = 150.90 # era  156.00
HGRE11_prc = 100.76 # era  104.97
HTMX11_prc = 145.84 # era  197.98
RBRP11_prc =  44.75 # era   50.29
TVRI11_prc =  90.31 # era   96.82
VISC11_prc = 100.12 # era  103.57
XPIN11_prc =  67.70 # era   72.66
XPLG11_prc =  92.89 # era   96.94

solver.Add(FATN11 * FATN11_prc <= valor_equi_max)
solver.Add(HGLG11 * HGLG11_prc <= valor_equi_max)
solver.Add(HGRE11 * HGRE11_prc <= valor_equi_max)
solver.Add(HTMX11 * HTMX11_prc <= valor_equi_max)
solver.Add(RBRP11 * RBRP11_prc <= valor_equi_max)
solver.Add(TVRI11 * TVRI11_prc <= valor_equi_max)
solver.Add(VISC11 * VISC11_prc <= valor_equi_max)
solver.Add(XPIN11 * XPIN11_prc <= valor_equi_max)
solver.Add(XPLG11 * XPLG11_prc <= valor_equi_max)

solver.Add(FATN11 * FATN11_prc >= valor_equi_min)
solver.Add(HGLG11 * HGLG11_prc >= valor_equi_min)
solver.Add(HGRE11 * HGRE11_prc >= valor_equi_min)
solver.Add(HTMX11 * HTMX11_prc >= valor_equi_min)
solver.Add(RBRP11 * RBRP11_prc >= valor_equi_min)
solver.Add(TVRI11 * TVRI11_prc >= valor_equi_min)
solver.Add(VISC11 * VISC11_prc >= valor_equi_min)
solver.Add(XPIN11 * XPIN11_prc >= valor_equi_min)
solver.Add(XPLG11 * XPLG11_prc >= valor_equi_min)

solver.Add(                \
     FATN11 * FATN11_prc + \
     HGLG11 * HGLG11_prc + \
     HGRE11 * HGRE11_prc + \
     HTMX11 * HTMX11_prc + \
     RBRP11 * RBRP11_prc + \
     TVRI11 * TVRI11_prc + \
     VISC11 * VISC11_prc + \
     XPIN11 * XPIN11_prc + \
     XPLG11 * XPLG11_prc <= valor_maximo
)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x120744780> >

In [81]:
import statistics

lista = [
    1.66,
    2.19,
    1.98,
    1.41,
    2.30,
    2.13,
    2.15,
    2.37,
    3.42,
    3.94,
    1.23,
    0.98
]

med = statistics.mean(lista)
med

lista = [
    0.63,
    1.90,
    1.74,
    1.88,
    1.87,
    2.02,
    2.85,
    3.70,
    3.70,
    3.25,
    0.86,
    2.60
]

med = statistics.mean(lista)
med

2.25

In [82]:
# proventos

FATN11_dy = 0.80 # era 0.78
HGLG11_dy = 1.10 # era 1.10
HGRE11_dy = 0.85 # era 0.64
HTMX11_dy = 2.14 # era 2.25
RBRP11_dy = 0.41 # era 0.35
TVRI11_dy = 1.03 # era 0.90
VISC11_dy = 0.80 # era 0.97
XPIN11_dy = 0.74 # era ?
XPLG11_dy = 0.82 # era ?

solver.Maximize(
    FATN11 * FATN11_dy + \
    HGLG11 * HGLG11_dy + \
    HGRE11 * HGRE11_dy + \
    HTMX11 * HTMX11_dy + \
    RBRP11 * RBRP11_dy + \
    TVRI11 * TVRI11_dy + \
    VISC11 * VISC11_dy + \
    XPIN11 * XPIN11_dy + \
    XPLG11 * XPLG11_dy
)

In [83]:
status = solver.Solve()

In [84]:
if status == pywraplp.Solver.OPTIMAL:
    print(f"Solução:")
    print(f"Calculado em {solver.wall_time():.2f}ms utilizando {solver.iterations()} iterações")
    print('-' * 20)
    FATN11_total = FATN11.solution_value()
    HGLG11_total = HGLG11.solution_value()
    HGRE11_total = HGRE11.solution_value()
    HTMX11_total = HTMX11.solution_value()
    RBRP11_total = RBRP11.solution_value()
    TVRI11_total = TVRI11.solution_value()
    VISC11_total = VISC11.solution_value()
    XPIN11_total = XPIN11.solution_value()
    XPLG11_total = XPLG11.solution_value()

    FATN11_buy = FATN11_total - FATN11_min_qtd
    HGLG11_buy = HGLG11_total - HGLG11_min_qtd
    HGRE11_buy = HGRE11_total - HGRE11_min_qtd
    HTMX11_buy = HTMX11_total - HTMX11_min_qtd
    RBRP11_buy = RBRP11_total - RBRP11_min_qtd
    TVRI11_buy = TVRI11_total - TVRI11_min_qtd
    VISC11_buy = VISC11_total - VISC11_min_qtd
    XPIN11_buy = XPIN11_total - XPIN11_min_qtd
    XPLG11_buy = XPLG11_total - XPLG11_min_qtd

    FATN11_amount = FATN11_buy * FATN11_prc
    HGLG11_amount = HGLG11_buy * HGLG11_prc
    HGRE11_amount = HGRE11_buy * HGRE11_prc
    HTMX11_amount = HTMX11_buy * HTMX11_prc
    RBRP11_amount = RBRP11_buy * RBRP11_prc
    TVRI11_amount = TVRI11_buy * TVRI11_prc
    VISC11_amount = VISC11_buy * VISC11_prc
    XPIN11_amount = XPIN11_buy * XPIN11_prc
    XPLG11_amount = XPLG11_buy * XPLG11_prc

    FATN11_tot_amount = (FATN11_min_qtd + FATN11_buy) * FATN11_prc
    HGLG11_tot_amount = (HGLG11_min_qtd + HGLG11_buy) * HGLG11_prc
    HGRE11_tot_amount = (HGRE11_min_qtd + HGRE11_buy) * HGRE11_prc
    HTMX11_tot_amount = (HTMX11_min_qtd + HTMX11_buy) * HTMX11_prc
    RBRP11_tot_amount = (RBRP11_min_qtd + RBRP11_buy) * RBRP11_prc
    TVRI11_tot_amount = (TVRI11_min_qtd + TVRI11_buy) * TVRI11_prc
    VISC11_tot_amount = (VISC11_min_qtd + VISC11_buy) * VISC11_prc
    XPIN11_tot_amount = (XPIN11_min_qtd + XPIN11_buy) * XPIN11_prc
    XPLG11_tot_amount = (XPLG11_min_qtd + XPLG11_buy) * XPLG11_prc

    print(f"FATN11: {FATN11_total} = {FATN11_min_qtd} (now) + {FATN11_buy} (buy) \t | + R$ {FATN11_amount:.2f} | R$ {FATN11_tot_amount:.2f}")
    print(f"HGLG11: {HGLG11_total} = {HGLG11_min_qtd} (now) + {HGLG11_buy} (buy) \t | + R$ {HGLG11_amount:.2f} | R$ {HGLG11_tot_amount:.2f}")
    print(f"HGRE11: {HGRE11_total} = {HGRE11_min_qtd} (now) + {HGRE11_buy} (buy) \t | + R$ {HGRE11_amount:.2f} | R$ {HGRE11_tot_amount:.2f}")
    print(f"HTMX11: {HTMX11_total} = {HTMX11_min_qtd} (now) + {HTMX11_buy} (buy) \t | + R$ {HTMX11_amount:.2f} | R$ {HTMX11_tot_amount:.2f}")
    print(f"RBRP11: {RBRP11_total} = {RBRP11_min_qtd} (now) + {RBRP11_buy} (buy)    | + R$ {RBRP11_amount:.2f} | R$ {RBRP11_tot_amount:.2f}")
    print(f"TVRI11: {TVRI11_total} = {TVRI11_min_qtd} (now) + {TVRI11_buy} (buy) \t | + R$ {TVRI11_amount:.2f} | R$ {TVRI11_tot_amount:.2f}")
    print(f"VISC11: {VISC11_total} = {VISC11_min_qtd} (now) + {VISC11_buy} (buy) \t | + R$ {VISC11_amount:.2f} | R$ {VISC11_tot_amount:.2f}")
    print(f"XPIN11: {XPIN11_total} = {XPIN11_min_qtd} (now) + {XPIN11_buy} (buy) \t | + R$ {XPIN11_amount:.2f} | R$ {XPIN11_tot_amount:.2f}")
    print(f"XPLG11: {XPLG11_total} = {XPLG11_min_qtd} (now) + {XPLG11_buy} (buy) \t | + R$ {XPLG11_amount:.2f} | R$ {XPLG11_tot_amount:.2f}")
    
    print('-' * 20)
    total = FATN11_prc * FATN11.solution_value() + \
            HGLG11_prc * HGLG11.solution_value() + \
            HGRE11_prc * HGRE11.solution_value() + \
            HTMX11_prc * HTMX11.solution_value() + \
            RBRP11_prc * RBRP11.solution_value() + \
            TVRI11_prc * TVRI11.solution_value() + \
            VISC11_prc * VISC11.solution_value() + \
            XPIN11_prc * XPIN11.solution_value() + \
            XPLG11_prc * XPLG11.solution_value()
    print(f"Valor de proventos estimado maximidado: R$ {solver.Objective().Value():.2f}")
    print(f"Total investido neste cenário: R$ {total:.2f} (+{(total - 44_744.61):.2f})")
    print(f"Yield total neste cenário: {(solver.Objective().Value() / total)*100:.2f}%")

else:
    print(f"O Solver não conseguiu encontrar uma solução ótima.") 

Solução:
Calculado em 84.00ms utilizando 3 iterações
--------------------
FATN11: 133.0 = 62 (now) + 71.0 (buy) 	 | + R$ 5335.65 | R$ 9994.95
HGLG11: 60.0 = 35 (now) + 25.0 (buy) 	 | + R$ 3772.50 | R$ 9054.00
HGRE11: 91.0 = 45 (now) + 46.0 (buy) 	 | + R$ 4634.96 | R$ 9169.16
HTMX11: 68.0 = 33 (now) + 35.0 (buy) 	 | + R$ 5104.40 | R$ 9917.12
RBRP11: 214.0 = 102 (now) + 112.0 (buy)    | + R$ 5012.00 | R$ 9576.50
TVRI11: 110.0 = 56 (now) + 54.0 (buy) 	 | + R$ 4876.74 | R$ 9934.10
VISC11: 91.0 = 48 (now) + 43.0 (buy) 	 | + R$ 4305.16 | R$ 9110.92
XPIN11: 146.0 = 76 (now) + 70.0 (buy) 	 | + R$ 4739.00 | R$ 9884.20
XPLG11: 98.0 = 54 (now) + 44.0 (buy) 	 | + R$ 4087.16 | R$ 9103.22
--------------------
Valor de proventos estimado maximidado: R$ 857.51
Total investido neste cenário: R$ 85744.17 (+40999.56)
Yield total neste cenário: 1.00%


| PAPEL | TIPO | VALOR UN. | DATA | QTD. COMPRADA | QTD. SUGERIDA |
|:-:|:-:|:-:|:-:|:-:|:-:|
| FATN11	| FII	| R$ ( 89,69)	| 2024/10/11	|  63  |  66 |
| HGLG11	| FII	| R$ (156,22)	| 2024/10/11	|  36  |  38 |
| HGRE11	| FII	| R$ (104,93)	| 2024/10/11	|  46  |  57 |
| HTMX11	| FII	| R$ (196,68)	| 2024/10/11	|  34  |  30 |
| RBRP11	| FII	| R$ ( 50,16)	| 2024/10/11	| 103  | 119 |
| TVRI11	| FII	| R$ ( 96,84)	| 2024/10/11	|  57  |  61 |
| VISC11	| FII	| R$ (103,81)	| 2024/10/11	|  49  |  57 |
| XPIN11	| FII	| R$ ( 72,54)	| 2024/10/11	|  77  |  82 |
| XPLG11	| FII	| R$ ( 96,50)	| 2024/10/11	|  55  |  61 |